In [100]:
using ITensors, ITensorMPS
using LinearAlgebra
using Plots
using HDF5

N = 8

f4 = h5open("Basis states Final/MPO_$N.h5","r")
W1 = read(f4,"W",MPO)
close(f4)

# Assuming `siteinds(W1)` gives the indices of the MPO
all_sites = siteinds(W1)

# Consider only physical sites
s = [pair[2] for pair in all_sites]

# println(s)

8-element Vector{Index{Int64}}:
 (dim=2|id=677|"Qubit,Site,n=1")
 (dim=2|id=758|"Qubit,Site,n=2")
 (dim=2|id=316|"Qubit,Site,n=3")
 (dim=2|id=834|"Qubit,Site,n=4")
 (dim=2|id=910|"Qubit,Site,n=5")
 (dim=2|id=834|"Qubit,Site,n=6")
 (dim=2|id=774|"Qubit,Site,n=7")
 (dim=2|id=708|"Qubit,Site,n=8")

In [101]:
# Taking the input 

x = range(0, stop=2π, length=2^(N))
input_array = cos.(x)


array = input_array / norm(input_array) # Input
ITensors.disable_warn_order()

cutoff1 = 1E-24
maxdim1 = 10


T = ITensor(array,s)

ψ = MPS(T,s;cutoff=cutoff1,maxdim=maxdim1)

orthogonalize!(ψ, 1) # Orthogonalize Psi 

start = time()

result = contract(W1, ψ)
end_time = time()
time1 = end_time - start

println(time1)

0.012466907501220703


In [102]:
using FFTW

start2 = time()
x2 = fft(array)
end2 = time()

time2 = end2 - start2
println(time2)

0.0008208751678466797


# Check Output 

In [104]:

inner_products = []
MPS2 = []

for i in 1:2^N
    f2 = h5open("Basis states Final/MPS_$N/MPS_create_$i.h5","r")
    mps1 = read(f2,"M",MPS)
    push!(MPS2,mps1)
    close(f2)
end

for i in 1:(2^N)
    push!(inner_products, inner(MPS2[i], result))
    print(inner(MPS2[i], result)*2^(N/2))
    println()
end


0.08821621827804672 + 8.007977567406804e-15im
11.312637297815002 + 0.13883391565100936im
-0.029702393547859987 - 0.0007291528349565411im
-0.0111145606573291 - 0.00040937350978934066im
-0.005920880686734217 - 0.00029087421600755884im
-0.0036964767920609006 - 0.00022709805153437628im
-0.0025316625887280924 - 0.00018674665166915536im
-0.00184347687020024 - 0.00015875073626916026im
-0.0014022967236804857 - 0.0001381141713956789im
-0.0011024510439008412 - 0.00012225951840810336im
-0.000889405126161656 - 0.00010969766373339607im
-0.0007320008007956385 - 9.941762110806319e-5im
-0.0006126214694225443 - 9.087381186068352e-5im
-0.000520087843339552 - 8.368283963997989e-5im
-0.00044719898248398484 - 7.759638633979977e-5im
-0.00038799553267972225 - 7.2239103332238e-5im
-0.0003400305836794849 - 6.763628958914149e-5im
-0.0002997986727891268 - 6.346786590215882e-5im
-0.0002650464242225837 - 5.951815594503899e-5im
-0.00023805432824788636 - 5.653421367886434e-5im
-0.00021501459102190503 - 5.38583484112

In [105]:
for i in 1:2^3
    println(x2[i])
end

0.08821621827824525 + 0.0im
11.312637297814938 + 0.1388339156511108im
-0.029702393548572584 - 0.0007291528349543864im
-0.011114560657359013 - 0.00040937350980998496im
-0.005920880686223092 - 0.00029087421597499743im
-0.0036964767915395806 - 0.00022709805147438884im
-0.0025316625892162583 - 0.0001867466517001828im
-0.00184347687019808 - 0.00015875073636385448im


# Code to generate basis states to measure

In [99]:
function generate_basis_states(n)
    basis_states = []
    for i in 0:(2^n - 1)
        binary_str = bitstring(i)[end-n+1:end]  # Get last n bits of bitstring
        push!(basis_states, collect(binary_str))  # Collect splits string into array of characters
    end
    return basis_states
end

MPS_states = []
basis_states = generate_basis_states(N)

for i in 1:(2^N)
    push!(MPS_states, MPS(s,string.(basis_states[i])))
end

using HDF5
f = 0 

for (i, mps) in enumerate(MPS_states)
    f = h5open("Basis states Final/MPS_$N/MPS_create_$i.h5","w")
    write(f,"M", mps)
end
close(f)